**Importing Libraries**

In [ ]:
import tensorflow as tf
from keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator # Data Augumentation:  increasing the training set
                                                # by creating modified copies of a dataset using existing data.
from keras.models import Model
# Models
from keras.applications import InceptionV3
from keras.applications import MobileNetV2
from keras.applications import Xception

# **Data preparation**

In [ ]:
batchsize = 8
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=0.2,shear_range=0.2,zoom_range=0.2,
                                   width_shift_range=0.2,height_shift_range=0.2, validation_split=0.2)

train_data = train_datagen.flow_from_directory(r'/content/drive/MyDrive/AI Project/Eyes_Data/train',
                                               target_size=(80,80), batch_size=batchsize, class_mode='categorical',
                                               subset='training')

validation_data = train_datagen.flow_from_directory(r'/content/drive/MyDrive/AI Project/Eyes_Data/train',
                                               target_size=(80,80), batch_size=batchsize, class_mode='categorical',
                                               subset='validation')


Found 20784 images belonging to 2 classes.
Found 5196 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'/content/drive/MyDrive/AI Project/Eyes_Data/test',
                                               target_size=(80,80), batch_size=batchsize, class_mode='categorical')


Found 6544 images belonging to 2 classes.


# **Data Preparation 2**

In [ ]:
batchsize = 8
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=0.2,shear_range=0.2,zoom_range=0.2,
                                   width_shift_range=0.2,height_shift_range=0.2, validation_split=0.2)

train_data = train_datagen.flow_from_directory(r'/content/drive/MyDrive/AI Project/Eyes_Data/train',
                                               target_size=(80,80), batch_size=batchsize, class_mode='binary',
                                               subset='training')

validation_data = train_datagen.flow_from_directory(r'/content/drive/MyDrive/AI Project/Eyes_Data/train',
                                               target_size=(80,80), batch_size=batchsize, class_mode='binary',
                                               subset='validation')

Found 20784 images belonging to 2 classes.
Found 5196 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'/content/drive/MyDrive/AI Project/Eyes_Data/test',
                                               target_size=(80,80), batch_size=batchsize, class_mode='binary')

Found 6544 images belonging to 2 classes.


# **Deep learning InceptionV3 2**

In [ ]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(1, activation='sigmoid')(hmodel)

model = Model(inputs=bmodel.input, outputs=hmodel)
for layer in bmodel.layers:
    layer.trainable = False


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
checkpoint = ModelCheckpoint(r'/content/drive/MyDrive/AI Project/models/InceptionV3_2_model.h5',
                             monitor='val_loss', save_best_only=True, verbose=3)

earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=3)

callbacks =  [checkpoint, earlystop, learning_rate]


In [ ]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_data, steps_per_epoch=train_data.samples//batchsize,
                    validation_data=validation_data, validation_steps=validation_data.samples//batchsize,
                    callbacks=callbacks, epochs=14)




Epoch 1/14
2598/2598 [==============================] - ETA: 0s - loss: 0.1298 - accuracy: 0.9541
Epoch 1: val_loss improved from inf to 0.73570, saving model to /content/drive/MyDrive/AI Project/models/InceptionV3_2_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2598/2598 [==============================] - 7361s 3s/step - loss: 0.1298 - accuracy: 0.9541 - val_loss: 0.7357 - val_accuracy: 0.8030 - lr: 0.0010
Epoch 2/14
2598/2598 [==============================] - ETA: 0s - loss: 0.0965 - accuracy: 0.9657
Epoch 2: val_loss did not improve from 0.73570
2598/2598 [==============================] - 494s 190ms/step - loss: 0.0965 - accuracy: 0.9657 - val_loss: 0.7886 - val_accuracy: 0.8153 - lr: 0.0010
Epoch 3/14
2598/2598 [==============================] - ETA: 0s - loss: 0.0863 - accuracy: 0.9687
Epoch 3: val_loss did not improve from 0.73570
2598/2598 [==============================] - 492s 189ms/step - loss: 0.0863 - accuracy: 0.9687 - val_loss: 0.8668 - val_accuracy: 0.8238 - lr: 0.0010
Epoch 4/14
2598/2598 [==============================] - ETA: 0s - loss: 0.0829 - accuracy: 0.9709
Epoch 4: val_loss did not improve from 0.73570

Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
2598/2598 [============================

# Model Evaluation

In [ ]:
loss_test, acc_test = model.evaluate(test_data)
print(acc_test)
print(loss_test)


818/818 [==============================] - 1052s 1s/step - loss: 0.3730 - accuracy: 0.8950
0.3729638159275055
0.8950183391571045


# **Deep learning InceptionV3**

In [ ]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs=hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [ ]:
checkpoint = ModelCheckpoint(r'/content/drive/MyDrive/AI Project/models/InceptionV3_model.h5',
                             monitor='val_loss', save_best_only=True, verbose=3)

earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=3)

callbacks =  [checkpoint, earlystop, learning_rate]


In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data, steps_per_epoch=train_data.samples//batchsize,
                    validation_data=validation_data, validation_steps=validation_data.samples//batchsize,
                    callbacks=callbacks, epochs=14)


<ipython-input-8-69ff36be1931>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data, steps_per_epoch=train_data.samples//batchsize,


Epoch 1/14
2598/2598 [==============================] - ETA: 0s - loss: 0.1313 - accuracy: 0.9546
Epoch 1: val_loss improved from inf to 0.66956, saving model to /content/drive/MyDrive/AI Project/models/Inception_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2598/2598 [==============================] - 4659s 2s/step - loss: 0.1313 - accuracy: 0.9546 - val_loss: 0.6696 - val_accuracy: 0.8193 - lr: 0.0010
Epoch 2/14
2598/2598 [==============================] - ETA: 0s - loss: 0.0994 - accuracy: 0.9658
Epoch 2: val_loss improved from 0.66956 to 0.60248, saving model to /content/drive/MyDrive/AI Project/models/Inception_model.h5
2598/2598 [==============================] - 421s 162ms/step - loss: 0.0994 - accuracy: 0.9658 - val_loss: 0.6025 - val_accuracy: 0.8384 - lr: 0.0010
Epoch 3/14
2598/2598 [==============================] - ETA: 0s - loss: 0.0921 - accuracy: 0.9657
Epoch 3: val_loss did not improve from 0.60248
2598/2598 [==============================] - 417s 160ms/step - loss: 0.0921 - accuracy: 0.9657 - val_loss: 0.7534 - val_accuracy: 0.8382 - lr: 0.0010
Epoch 4/14
2598/2598 [==============================] - ETA: 0s - loss: 0.0816 - accuracy: 0.9716
Epoch 4: val_loss did not improve from 0.60248
2598/2598 [=========================

# Model Evaluation

In [ ]:
loss_test, acc_test = model.evaluate(test_data)
print(acc_test)
print(loss_test)


818/818 [==============================] - 906s 1s/step - loss: 0.2061 - accuracy: 0.9273
0.20614810287952423
0.9272615909576416


# **Deep learning MobileNet V2**

In [ ]:
bmodel = MobileNetV2(input_shape=(80,80,3), include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

model2 = Model(inputs=bmodel.input, outputs=hmodel)
for layer in bmodel.layers:
    layer.trainable = False


9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(r'/content/drive/MyDrive/AI Project/models/MobileNetV2_model.h5',
                             monitor='val_loss', save_best_only=True, verbose=3)

earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=3)

callbacks =  [checkpoint, earlystop, learning_rate]


In [ ]:
model2.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model2.fit(train_data, steps_per_epoch=train_data.samples//batchsize,
                    validation_data=validation_data, validation_steps=validation_data.samples//batchsize,
                    callbacks=callbacks, epochs=14)

Epoch 1/14
2598/2598 [==============================] - ETA: 0s - loss: 0.2960 - accuracy: 0.8923
Epoch 1: val_loss improved from inf to 0.81828, saving model to /content/drive/MyDrive/AI Project/models/MobileNetV2_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2598/2598 [==============================] - 7769s 3s/step - loss: 0.2960 - accuracy: 0.8923 - val_loss: 0.8183 - val_accuracy: 0.7741 - lr: 0.0010
Epoch 2/14
2598/2598 [==============================] - ETA: 0s - loss: 0.1857 - accuracy: 0.9224
Epoch 2: val_loss did not improve from 0.81828
2598/2598 [==============================] - 263s 101ms/step - loss: 0.1857 - accuracy: 0.9224 - val_loss: 1.2335 - val_accuracy: 0.7535 - lr: 0.0010
Epoch 3/14
2597/2598 [============================>.] - ETA: 0s - loss: 0.1658 - accuracy: 0.9299
Epoch 3: val_loss did not improve from 0.81828
2598/2598 [==============================] - 257s 99ms/step - loss: 0.1658 - accuracy: 0.9298 - val_loss: 0.9812 - val_accuracy: 0.7943 - lr: 0.0010
Epoch 4/14
2598/2598 [==============================] - ETA: 0s - loss: 0.1546 - accuracy: 0.9339
Epoch 4: val_loss did not improve from 0.81828

Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
2598/2598 [=============================

Model

In [ ]:
loss_test, acc_test = model2.evaluate(test_data)
print(acc_test)
print(loss_test)


818/818 [==============================] - 1893s 2s/step - loss: 0.6804 - accuracy: 0.8013
0.6803821921348572
0.8013447523117065


# **Deep learning Xception**

In [ ]:
bmodel = Xception(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

model3 = Model(inputs=bmodel.input, outputs=hmodel)
for layer in bmodel.layers:
    layer.trainable = False


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(r'/content/drive/MyDrive/AI Project/models/Xception_model.h5',
                             monitor='val_loss', save_best_only=True, verbose=3)

earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=3)

callbacks =  [checkpoint, earlystop, learning_rate]


In [ ]:
model3.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model3.fit(train_data, steps_per_epoch=train_data.samples//batchsize,
                    validation_data=validation_data, validation_steps=validation_data.samples//batchsize,
                    callbacks=callbacks, epochs=14)

Epoch 1/14
2598/2598 [==============================] - ETA: 0s - loss: 0.1688 - accuracy: 0.9465
Epoch 1: val_loss improved from inf to 0.99619, saving model to /content/drive/MyDrive/AI Project/models/Xception_model.h5
2598/2598 [==============================] - 939s 356ms/step - loss: 0.1688 - accuracy: 0.9465 - val_loss: 0.9962 - val_accuracy: 0.7854 - lr: 0.0010
Epoch 2/14
2598/2598 [==============================] - ETA: 0s - loss: 0.1196 - accuracy: 0.9617
Epoch 2: val_loss did not improve from 0.99619
2598/2598 [==============================] - 953s 367ms/step - loss: 0.1196 - accuracy: 0.9617 - val_loss: 1.0735 - val_accuracy: 0.8155 - lr: 0.0010
Epoch 3/14
2598/2598 [==============================] - ETA: 0s - loss: 0.1038 - accuracy: 0.9673
Epoch 3: val_loss did not improve from 0.99619
2598/2598 [==============================] - 949s 365ms/step - loss: 0.1038 - accuracy: 0.9673 - val_loss: 1.1994 - val_accuracy: 0.8032 - lr: 0.0010
Epoch 4/14
2598/2598 [=================

In [ ]:
loss_test, acc_test = model3.evaluate(test_data)
print(acc_test)
print(loss_test)


818/818 [==============================] - 214s 262ms/step - loss: 0.3682 - accuracy: 0.8927
0.3682295083999634
0.8927261829376221
